In [ ]:
#!pip install pyttsx3 pydub

  Using cached pydub-0.25.1-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pydub-0.25.1-py2.py3-none-any.whl (32 kB)


In [ ]:
import pyttsx3
import re
import os
from pydub import AudioSegment

# Initialize TTS engine
engine = pyttsx3.init()
voices = engine.getProperty('voices')

voice_mapping = {
    "Examiner": voices[0].id,
    "Mark": voices[1].id, 
    "Laura": voices[2].id
}

speech_rate = 150
engine.setProperty('rate', speech_rate)

# Read transcript
txt_path = "transcripts.txt"
with open(txt_path, "r", encoding="utf-8") as f:
    lines = f.readlines()

final_audio = AudioSegment.silent(duration=0)

for line in lines:
    line = line.strip()
    if not line:
        continue
    
    # Handle pauses
    if "(Pause" in line:
        match = re.search(r"Pause (\d+) seconds", line)
        if match:
            seconds = int(match.group(1))
            # Use AudioSegment.silent() instead of Silent
            final_audio += AudioSegment.silent(duration=seconds*1000)
        continue
    
    # Detect speaker
    speaker = None
    for sp in voice_mapping.keys():
        if line.startswith(sp + ":"):
            speaker = sp
            text = line[len(sp)+1:].strip()
            break
    else:
        speaker = "Examiner"
        text = line

    # Set speaker voice
    engine.setProperty('voice', voice_mapping[speaker])
    
    # Save temporary TTS audio
    temp_file = f"temp_{speaker}.mp3"
    engine.save_to_file(text, temp_file)
    engine.runAndWait()

    # Append to final audio
    segment = AudioSegment.from_file(temp_file, format="mp3")
    final_audio += segment
    os.remove(temp_file)

# Export final audio
output_path = "conversation_section1.mp3"
final_audio.export(output_path, format="mp3")
print(f"Audio generated: {output_path}")
